In [3]:
import boto3

***Create a Key-Pair for Secure Connection***

In [ ]:
ec2 = boto3.client('ec2', region_name='us-east-1')

ec2.describe_instances()

In [ ]:
resp = ec2.create_key_pair(KeyName='mlops-key-2')

In [9]:
file = open('creds/mlops-key-2.pem', 'w')
file.write(resp['KeyMaterial'])
file.close()

***Create EC2 instance***

In [10]:
response = ec2.run_instances(
    ImageId='ami-068c0051b15cdb816',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='mlops-key-2',
    # Define Volume
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/xvda',
            'Ebs': {
                'VolumeSize': 20,
                'DeleteOnTermination': True
            }
        }
    ]
)

In [ ]:
response

***Create Security Group***

In [12]:
response = ec2.create_security_group(
    GroupName='mlops-sg-2',
    Description='Security group for ML Ops project'
)

In [13]:
response

{'GroupId': 'sg-009d4775eb8b7ede4',
 'ResponseMetadata': {'RequestId': 'f9138591-605f-40f5-946f-b62ebdc1c3e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f9138591-605f-40f5-946f-b62ebdc1c3e3',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '373',
   'date': 'Sun, 28 Dec 2025 20:12:26 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [14]:
security_group_id = response['GroupId']

***Configure Security Group***

In [15]:
response = ec2.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

***Connect Security Group to EC2 Instance***

In [24]:
response = ec2.describe_instances()

for reservation in response['Reservations']:
    if reservation['Instances'][0]['KeyName'] == 'mlops-key-2':
        instance_id = reservation['Instances'][0]['InstanceId']
        old_sg_ids = [sg['GroupId'] for sg in reservation['Instances'][0]['SecurityGroups']]
        print(instance_id)
        print(old_sg_ids)

i-02c64c921183e3e36
['sg-06b523260f085aceb']


In [ ]:
response = ec2.describe_security_groups()

for sg in response['SecurityGroups']:
    if sg['GroupName'] == 'mlops-sg-2':
        sg_id = sg['GroupId']
        print(sg['GroupId'])

sg-009d4775eb8b7ede4


{'Description': 'Security group for ML Ops project',
 'GroupName': 'mlops-sg-2',
 'IpPermissions': [{'FromPort': 22,
   'IpProtocol': 'tcp',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'ToPort': 22,
   'UserIdGroupPairs': []}],
 'OwnerId': '683694378087',
 'GroupId': 'sg-009d4775eb8b7ede4',
 'IpPermissionsEgress': [{'IpProtocol': '-1',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'UserIdGroupPairs': []}],
 'VpcId': 'vpc-0aea73ec1e4850e1b'}

In [25]:
ec2.modify_instance_attribute(
    InstanceId=instance_id,
    Groups=[sg_id]
)

{'ResponseMetadata': {'RequestId': '65cb3e94-0d50-43a0-86e9-683c427d417e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '65cb3e94-0d50-43a0-86e9-683c427d417e',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Mon, 29 Dec 2025 14:14:10 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

***Start, Stop, Terminate Instances***

In [26]:
import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=[instance_id])
        current_status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if current_status == target_status:
            print(f"Instance is in '{target_status}' status.")
            break
        else:
            print(f"Instance is in {current_status}' status. Waiting...")

        time.sleep(10)

def start_instance(instance_id):
    print("Starting instance...")
    ec2.start_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, 'running')


start_instance(instance_id)

Starting instance...
Instance is in stopped' status. Waiting...
Instance is in 'running' status.


In [27]:
def stop_instance(instance_id):
    print("Stopping instance...")
    ec2.stop_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, 'stopped')

stop_instance(instance_id)

Stopping instance...
Instance is in running' status. Waiting...
Instance is in stopping' status. Waiting...
Instance is in stopping' status. Waiting...
Instance is in stopping' status. Waiting...
Instance is in stopping' status. Waiting...
Instance is in stopping' status. Waiting...
Instance is in 'stopped' status.


In [28]:
def terminate_instance(instance_id):
    print("Terminating instance...")
    ec2.terminate_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, 'terminated')

terminate_instance(instance_id)

Terminating instance...
Instance is in stopped' status. Waiting...
Instance is in 'terminated' status.
